In [1]:
import arcpy, os, re, datetime, sys, glob 
import multiprocessing as mp

In [2]:
inFeatureClass = r'C:\Temp\C2VSimFG-BETA_GIS\Shapefiles\C2VSimFG_Elements.shp'
outUnits = 'inches'
NSPRN = 1
NFQRN = 0

In [3]:
def FACTRN(outUnits):
    if outUnits == 'feet':
        return 1.0
    if outUnits == 'inches':
        return 1.0/12.0
    if outUnits == 'meters':
        return 12.0/39.37
    if outUnits == 'millimeters':
        return 0.001 * 39.37/12.0

In [4]:
        def PrecipSpecs(NRAIN, FACTRN, NSPRN, NFQRN):
            string = """
C*******************************************************************************
C                         Rainfall Data Specifications
C
C   NRAIN ;  Number of rainfall stations (or pathnames if DSS files are used)
C             used in the model 
C   FACTRN;  Conversion factor for rainfall rate 
C             It is used to convert only the spatial component of the unit; 
C             DO NOT include the conversion factor for time component of the unit.
C             * e.g. Unit of rainfall rate listed in this file = INCHES/MONTH
C                    Consistent unit used in simulation        = FEET/DAY 
C                    Enter FACTRN (INCHES/MONTH -> FEET/MONTH) = 8.33333E-02 
C                     (conversion of MONTH -> DAY is performed automatically) 
C   NSPRN ;  Number of time steps to update the precipitation data
C             * Enter any number if time-tracking option is on
C   NFQRN ;  Repetition frequency of the precipitation data 
C             * Enter 0 if full time series data is supplied
C             * Enter any number if time-tracking option is on
C   DSSFL ;  The name of the DSS file for data input (maximum 50 characters); 
C             * Leave blank if DSS file is not used for data input
C 
C-------------------------------------------------------------------------------
C         VALUE                                      DESCRIPTION
C-------------------------------------------------------------------------------
          {:<43}/ NRAIN 
          {:<43}/ FACTRN  (in/month -> ft/month)         
          {:<43}/ NSPRN
          {:<43}/ NFQRN
                                                     / DSSFL"""
            return string.format(NRAIN, FACTRN, NSPRN, NFQRN)

In [5]:
featureCount = int(arcpy.GetCount_management(inFeatureClass)[0])

In [6]:
def PrecipData(NRAIN):
    string = """
C-------------------------------------------------------------------------------
C                             Rainfall Data 
C                         (READ FROM THIS FILE)
C
C   List the rainfall rates for each of the rainfall station below, if it will 
C   not be read from a DSS file (i.e. DSSFL is left blank above).
C
C   ITRN ;   Time 
C   ARAIN;   Rainfall rate at the corresponding rainfall station; [L/T]
C
C-------------------------------------------------------------------------------     
C   ITRN          ARAIN(1)  ARAIN(2)  ARAIN(3) ...
C   TIME        {}
C-------------------------------------------------------------------------------
"""
    return string.format('{:>10}'*NRAIN).format(*[i+1 for i in range(NRAIN)])

In [7]:
def LastDayOfMonth(inDate):
    ''' generates a date for the last day of a given month '''
    nextMonth = inDate.replace(day=28) + datetime.timedelta(days=4)
    return nextMonth - datetime.timedelta(days=nextMonth.day)

In [8]:
inRaster = r'F:\ppt\1909\prism_ppt_us_30s_190905_clip.bil'

In [9]:
def ParseDateFromFileName(inFileName, fmt="%m/%d/%Y_24:00"):
    ''' use regex to parse dates in YYYYMM, YYYYM, YYYY_MM, or YYYY_M format and return the date as a string in the desired format '''

    # find date between 1900 and 2099 within basename of provided file
    expr = re.findall(r"19\d{2}0?[1-9](?!\d+)|19\d{2}1[0-2](?!\d+)|20\d{2}0?[1-9](?!\d+)|20\d{2}1[0-2](?!\d+)|19\d{2}_0?[1-9](?!\d+)|19\d{2}_1[0-2](?!\d+)|20\d{2}_0?[1-9](?!\d+)|20\d{2}_1[0-2](?!\d+)", os.path.basename(inFileName))[0]

    # determine the format of the expression and parse accordingly
    if "_" not in expr:
        fileDate = datetime.datetime.strptime(expr, "%Y%m")
    else:
        fileDate = datetime.datetime.strptime(expr, "%Y_%m")
    
    modelDate = datetime.datetime.strftime(LastDayOfMonth(fileDate), fmt)

    return modelDate

try:
    modelDate = ParseDateFromFileName(inFileName=inRaster, fmt='%m/%d/%Y_24:00')
except IndexError:
    exc = sys.exc_info()
    print("{0}: date could not be read from filename: {1}".format(exc[0].__name__,testRaster))
    print("files should have dates of the following format:\n" + \
          "    1. Years should be between 1900 and 2099\n" + \
          "    2. Format:\n" + \
          "        a. YYYYMM\n" + \
          "        b. YYYYM\n" + \
          "        c. YYYY_MM\n" + \
          "        d. YYYY_M\n")
    sys.exit(1)

print(modelDate)

05/31/1909_24:00


In [10]:
def GetAllRastersFromFolders(inWorkspace):
    ''' generates a list of all rasters in subfolders of the parent directory '''
    listDir = os.listdir(inWorkspace)
    listRasters = []
    
    # add rasters to list from parent directory
    rasterList = glob.glob(os.path.join(inWorkspace, '*.bil'))
    if len(rasterList) > 0:
        listRasters = [os.path.join(inWorkspace, raster) for raster in rasterList]
        
    # only do if the parent directory has subdirectories within it
    if len(listDir) > 0:
        dirPaths = [os.path.join(inWorkspace, dir) for dir in listDir]
        for dirPath in dirPaths:
            rasterList = glob.glob(os.path.join(dirPath, '*.bil'))
            if len(listRasters) > 0:
                listRasters.extend([os.path.join(dirPath, raster) for raster in rasterList])
            else:
                listRasters = [os.path.join(dirPath, raster) for raster in rasterList]
    
    return sorted(listRasters)

In [11]:
def GetAllRastersFromFolders2(inWorkspace):
    ''' generates a list of all rasters in subfolders of the parent directory '''
    listDir = os.listdir(inWorkspace)
    listRasters = []
    
    # add rasters to list from parent directory
    arcpy.env.workspace = inWorkspace
    rasterList = arcpy.ListRasters('*', 'All')
    if len(rasterList) > 0:
        listRasters = [os.path.join(inWorkspace, raster) for raster in rasterList]
        
    # only do if the parent directory has subdirectories within it
    if len(listDir) > 0:
        dirPaths = [os.path.join(inWorkspace, dir) for dir in listDir]
        for dirPath in dirPaths:
            arcpy.env.workspace = dirPath
            rasterList = arcpy.ListRasters('*', 'All')
            if len(listRasters) > 0:
                listRasters.extend([os.path.join(dirPath, raster) for raster in rasterList])
            else:
                listRasters = [os.path.join(dirPath, raster) for raster in rasterList]
    
    return sorted(listRasters)

In [12]:
inWorkspace = r'C:\Temp\Test\Clipped'

In [13]:
%time rasterList = GetAllRastersFromFolders(inWorkspace)

Wall time: 11 ms


In [14]:
%time rasterList2 = GetAllRastersFromFolders2(inWorkspace)

Wall time: 1.59 s


In [15]:
valueRaster = r'C:\Temp\Test\Clipped\prism_ppt_us_30s_201509_clip.bil'
#valueRaster = r'C:\Temp\C2VSimFG-BETA_GIS\Shapefiles\C2VSimFG_Elements_SmallWatersheds.shp'
arcpy.CheckOutExtension("Spatial")

# what error would occur if the dataset provided is not a raster?

def GetPropertiesFromRaster(inRaster):
    ''' returns raster properties from raster object '''
    # Instantiate a raster object for the raster to access its properties
    rasterDataset = arcpy.sa.Raster(inRaster)
          
    # Get properties from raster object
    inRasterName = rasterDataset.name
    numRows = rasterDataset.height
    numCols = rasterDataset.width
    rastExtent = rasterDataset.extent
    xMin = rastExtent.XMin
    yMin = rastExtent.YMin
    xMax = rastExtent.XMax
    yMax = rastExtent.YMax
    
    del rasterDataset
    
    return inRasterName, numRows, numCols, xMin, yMin, xMax, yMax

try:
    inRasterName, numRows, numCols, xMin, yMin, xMax, yMax = GetPropertiesFromRaster(valueRaster)
except RuntimeError:
    ex = sys.exc_info()
    try:
        desc = arcpy.Describe(valueRaster)
    except OSError:
        ex = sys.exc_info()
        print("{0}:The input to the function does not exist or could not be found.".format(ex[0].__name__))
    else:
        if desc.dataType != "RasterDataset":
            print("{0}: The input to the function is not a raster".format(ex[0].__name__))
        else:
            print("{0}: The input to the function does not exist or is not supported.".format(ex[0].__name__))


In [17]:
def CreateFishnetFeature(inRaster, outWorkspace):
    ''' creates a polyline fishnet feature class and returns it's name '''
    
    # Get properties from raster dataset for use in generating fishnet
    inRasterName, numRows, numCols, xMin, yMin, xMax, yMax = GetPropertiesFromRaster(inRaster)
    
    # Use the input raster name to generate a default output name for the fishnet feature class
    inRasterBaseName = os.path.splitext(inRasterName)[0]
    if "clip" in inRasterBaseName:
        outFishnetFeatureName = "{0}.shp".format(inRasterBaseName.replace("clip", "fishnet"))
    else:
        outFishnetFeatureName = "{0}_fishnet.shp".format(inRasterBaseName)
    
    outFishnetFeature = os.path.join(outWorkspace, outFishnetFeatureName)
    
    # Set origin and rotation of fishnet using extent properties
    # rotation is set by the difference in the x-values in the origin coordinate and the y-axis coordinate
    originCoordinate = "{0} {1}".format(xMin, yMin)
    yAxisCoordinate = "{0} {1}".format(xMin, yMax)
    
    # Create fishnet using clipped rasters
    arcpy.CreateFishnet_management(outFishnetFeature, originCoordinate, yAxisCoordinate, 0, 0, numRows, numCols, "#", "NO_LABELS", inRaster, "POLYLINE")
    
    return outFishnetFeature

In [16]:
inRasterName, inRasterExt = os.path.splitext(os.path.basename(valueRaster))
print(inRasterName)
print(inRasterExt)

prism_ppt_us_30s_201509_clip
.bil


In [17]:
def ConvertRasterToPoints(inRaster, outWorkspace):
    ''' converts a raster to points and returns the name of the point feature class '''
    
    # Use the input raster name to generate a default output name
    inRasterName = os.path.splitext(os.path.basename(inRaster))[0]
    outPointFeatureName = "{0}.shp".format(inRasterName)
    outPointFeature = os.path.join(outWorkspace, outPointFeatureName)
    
    # Convert raster to point feature class
    arcpy.RasterToPoint_conversion(inRaster, outPointFeature, "Value")
    
    return outPointFeature  

In [18]:
def ClipRaster(inRaster, clipFeature, outWorkspace):
    ''' clips a raster to the geometry of the boundary of the feature class provided and returns the name of the clipped raster '''
    
    # Use the input raster name to generate a default output name
    inRasterName, inRasterExt = os.path.splitext(os.path.basename(inRaster))
    outRasterName = "{0}_clip{1}".format(inRasterName, inRasterExt)
    outClipRaster = os.path.join(outWorkspace, outRasterName)
    
    # Clip raster to geometry of the feature class specified by the user
    arcpy.Clip_management(inRaster, "#", outClipRaster, clipFeature, "#", "ClippingGeometry")
    
    return outClipRaster

In [19]:
outWorkspace = r'C:\Temp\Test2'
outWorkspaceList = [outWorkspace for i in range(len(rasterList))]
inputList = tuple(zip(rasterList, outWorkspaceList))
print(inputList)

(('C:\\Temp\\Test\\Clipped\\prism_ppt_us_30s_201509_clip.bil', 'C:\\Temp\\Test2'), ('C:\\Temp\\Test\\Clipped\\prism_ppt_us_30s_201510_clip.bil', 'C:\\Temp\\Test2'), ('C:\\Temp\\Test\\Clipped\\prism_ppt_us_30s_201511_clip.bil', 'C:\\Temp\\Test2'), ('C:\\Temp\\Test\\Clipped\\prism_ppt_us_30s_201512_clip.bil', 'C:\\Temp\\Test2'))


In [20]:
def ConvertRasterToPoints2(inputValueList):
    ''' converts a raster to points and returns the name of the point feature class '''
    
    # Use the input raster name to generate a default output name
    inRaster, outWorkspace = inputValueList
    inRasterName = os.path.splitext(os.path.basename(inRaster))[0]
    outPointFeatureName = "{0}.shp".format(inRasterName)
    outPointFeature = os.path.join(outWorkspace, outPointFeatureName)
    
    # Convert raster to point feature class
    arcpy.RasterToPoint_conversion(inRaster, outPointFeature, "Value")
    
    return outPointFeature

In [ ]:
%%time
pool = mp.Pool(processes=4)
pointFeatureList = pool.starmap(ConvertRasterToPoints, inputList)
pool.close()

In [51]:
%%time
outLocation = r'C:\Temp\Test3'
for i, raster in enumerate(rasterList):
    ConvertRasterToPoints(raster, outLocation)

Wall time: 1min 8s


In [21]:
help(mp.Pool().starmap)

Help on method starmap in module multiprocessing.pool:

starmap(func, iterable, chunksize=None) method of multiprocessing.pool.Pool instance
    Like `map()` method but the elements of the `iterable` are expected to
    be iterables as well and will be unpacked as arguments. Hence
    `func` and (a, b) becomes func(a, b).

